# Your Name: Stephanie Buchanan

# import all packages 

In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import (OneHotEncoder, LabelEncoder, 
                                   StandardScaler, MinMaxScaler, RobustScaler)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve
import statsmodels.api as sm
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d

# Data Prepocessing

Description

The data are gathered from end of semester student evaluations for 463 courses taught by a sample of 94 professors from the University of Texas at Austin. In addition, six students rate the professors' physical appearance. The result is a data frame where each row contains a different course and each column has information on the course and the professor who taught that course. Usage

evals

Format

A data frame with 463 observations on the following 23 variables.

course_id

Variable identifying the course (out of 463 courses).

prof_id

Variable identifying the professor who taught the course (out of 94 professors).

score

Average professor evaluation score: (1) very unsatisfactory - (5) excellent.

rank

Rank of professor: teaching, tenure track, tenured.

ethnicity

Ethnicity of professor: not minority, minority.

gender

Gender of professor: female, male.

language

Language of school where professor received education: English or non-English.

age

Age of professor.

cls_perc_eval

Percent of students in class who completed evaluation.

cls_did_eval

Number of students in class who completed evaluation.

cls_students

Total number of students in class.

cls_level

Class level: lower, upper.

cls_profs

Number of professors teaching sections in course in sample: single, multiple.

cls_credits

Number of credits of class: one credit (lab, PE, etc.), multi credit.

bty_f1lower

Beauty rating of professor from lower level female: (1) lowest - (10) highest.

bty_f1upper

Beauty rating of professor from upper level female: (1) lowest - (10) highest.

bty_f2upper

Beauty rating of professor from second level female: (1) lowest - (10) highest.

bty_m1lower

Beauty rating of professor from lower level male: (1) lowest - (10) highest.

bty_m1upper

Beauty rating of professor from upper level male: (1) lowest - (10) highest.

bty_m2upper

Beauty rating of professor from second upper level male: (1) lowest - (10) highest.

bty_avg

Average beauty rating of professor.

pic_outfit

Outfit of professor in picture: not formal, formal.

pic_color

Color of professor's picture: color, black & white.

Source

Daniel S. Hamermesh, Amy Parker, Beauty in the classroom: instructors’ pulchritude and putative pedagogical productivity, Economics of Education Review, Volume 24, Issue 4, 2005. doi: 10.1016/j.econedurev.2004.07.013.

In [2]:
evals_df = pd.read_csv('evals.csv', index_col= 0)
evals_df.drop(['course_id', 'prof_id','language','cls_perc_eval','cls_did_eval', 'cls_credits',
               'bty_f1lower', 'bty_f1upper', 'bty_f2upper', 'bty_m1lower',
               'bty_m1upper', 'bty_m2upper', 'cls_students', 'pic_outfit', 'cls_profs',
              'cls_level', 'pic_color'], axis = 1, inplace = True)
evals_df.head()

,score,rank,ethnicity,gender,age,bty_avg
1,4.7,tenure track,minority,female,36,5.0
2,4.1,tenure track,minority,female,36,5.0
3,3.9,tenure track,minority,female,36,5.0
4,4.8,tenure track,minority,female,36,5.0
5,4.6,tenured,not minority,male,59,3.0


In [3]:
evals_df.columns

Index(['score', 'rank', 'ethnicity', 'gender', 'age', 'bty_avg'], dtype='object')

In [4]:
evals_df.isnull().sum()

score        0
rank         0
ethnicity    0
gender       0
age          0
bty_avg      0
dtype: int64

In [5]:
X_evals = evals_df.drop('rank', axis = 1).copy()
y_evals = (evals_df['rank'] == 'tenured').astype(int)
X_evals.head()

,score,ethnicity,gender,age,bty_avg
1,4.7,minority,female,36,5.0
2,4.1,minority,female,36,5.0
3,3.9,minority,female,36,5.0
4,4.8,minority,female,36,5.0
5,4.6,not minority,male,59,3.0


In [6]:
#change categorical data into numerical values
#Note: Label Encoded due to the binary categorical values.  
X_evals['ethnicity'] = LabelEncoder().fit_transform(X_evals['ethnicity'])
X_evals['gender'] = LabelEncoder().fit_transform(X_evals['gender'])
X_evals.head()

,score,ethnicity,gender,age,bty_avg
1,4.7,0,0,36,5.0
2,4.1,0,0,36,5.0
3,3.9,0,0,36,5.0
4,4.8,0,0,36,5.0
5,4.6,1,1,59,3.0


# Model Building and Evaluation 

In [7]:
# building the model and fitting the data
endog = y_evals
exog = sm.add_constant(X_evals)

log_reg = sm.Logit(endog, exog).fit()

Optimization terminated successfully.
         Current function value: 0.584703
         Iterations 5


In [8]:
print(log_reg.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.151     
Dependent Variable: rank             AIC:              553.4347  
Date:               2021-08-17 10:12 BIC:              578.2610  
No. Observations:   463              Log-Likelihood:   -270.72   
Df Model:           5                LL-Null:          -318.93   
Df Residuals:       457              LLR p-value:      2.9999e-19
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     5.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
const         -2.9811    1.1024  -2.7041  0.0068  -5.1418  -0.8203
score         -0.2860    0.2019  -1.4165  0.1566  -0.6817   0.1097
ethnicity      0.4353    0.2930   1.4856  0.1374  -0.1390   1.0095
gender         0.8222    0.216

In [9]:
coef = log_reg.params
coef

const       -2.981063
score       -0.285975
ethnicity    0.435258
gender       0.822162
age          0.079313
bty_avg     -0.062018
dtype: float64

# Conclusion 

The intended purpose of the model is to predict the tenured status, i.e., tenured = 1, not-tenured = 0, of a professor based on the professors evaluation score given by students, the ethnicity,gender, age and average beauty score of the professor.  A logistic regression model was fit, and the coefficicents of the logistic regression model can be interpreted as follows: 

* The score coefficient is -0.893262, meaning that if every other variable is held constant and the evaulation score given by the students to the professor is increased by 1, the log odds would decrease by 0.893262, and therefore the odds of the professor being tenured would increase by exp(-0.893262) = 0.40932.

* If the ethnicity goes from minority to not minority, the odds of the professor being tenured would increase by exp(0.513128) = 1.670508.
* If gender goes from female to male, the odds of the professor being tenured would increase by exp(0.817215) = 2.264185.
* If age is increased by 1, the odds of the professor being tenured would increase by exp(0.066788) = 1.06906881.
* If the average beauty score the professor received is increased by 1, the odds of the professor being tenured would increase by exp(-0.038029) = 0.9626850. 
* The coefficient is -2.981063, which means when x1 = x2 = x3 = x4 = x5 = 0, the log of the odds of the professor being tenured is -2.981063, or exp(-2.981063) = 0.05073886976949831.  

The model has a p value of 2.9999e-19 which is << 0.05 and is statistically significant.  Therefore, we reject the null hypothesis that all the coeffients are equal to 0.  Looking at the p-values for each individual attribute, P>|z|, the p-values > 0.05 are for the score, ethnicity and average beauty score features. Therefore, this is good reason to exclude these features from the logistic regression model. The pseudo R-squared value is 0.151, which is not particularly high, but should be compared to a model removing the features stated above. 

In [10]:
X_evals2 = X_evals.drop(['score', 'ethnicity', 'bty_avg'], axis = 1)

In [11]:
exog2 = sm.add_constant(X_evals2)

log_reg2 = sm.Logit(endog, exog2).fit()

Optimization terminated successfully.
         Current function value: 0.590134
         Iterations 5


In [12]:
print(log_reg2.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.143     
Dependent Variable: rank             AIC:              552.4640  
Date:               2021-08-17 10:12 BIC:              564.8771  
No. Observations:   463              Log-Likelihood:   -273.23   
Df Model:           2                LL-Null:          -318.93   
Df Residuals:       460              LLR p-value:      1.4279e-20
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     5.0000                                       
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
const     -4.3005     0.5725   -7.5114   0.0000   -5.4227   -3.1784
gender     0.7940     0.2096    3.7872   0.0002    0.3831    1.2049
age        0.0842     0.0119    7.0829   0.0000    0.0609    0.1075



The pseudo R-squared value with the unnecessary features is lower than in the more complex model : 0.143 vs. 0.151.  But, the p-value is lower and the p-values of the individual features are all <<0.05, and the most important measure in logistic regression the p-value since this is what is used to measure statistical significance.  